<a href="https://colab.research.google.com/github/ALejandroCh17/Tennis-Ball-Robot/blob/daniel_branch/model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/capstone_dataset_2/Tennis.v1i.yolov8.zip" -d "/content/drive/My Drive/capstone_dataset_3"

#Import Dependencies

In [4]:
import os

#Load the Dataset


In [5]:
# Base directory where train, test, and valid folders are located
base_dir = '/content/drive/My Drive/capstone_dataset_3'

In [6]:
# Dictionary to hold data associations for each dataset split
data_associations = {
    'train': {},
    'test': {},
    'valid': {}
}

In [7]:
# Function to associate images with labels within a specific folder
def associate_images_with_labels(image_folder, label_folder):
    associations = {}
    for image_file in os.listdir(image_folder):
        if image_file.endswith('.jpg'):  # Confirm it's an image file
            # Construct the label filename by changing the extension
            label_file = image_file.replace('.jpg', '.txt')
            # Build the full path to the label file
            label_file_path = os.path.join(label_folder, label_file)
            # Check if the corresponding label file exists
            if os.path.exists(label_file_path):
                # Save the association
                associations[image_file] = label_file
            else:
                print(f"Warning: Label file for {image_file} not found.")
    return associations

In [8]:
# Iterate through each dataset split and populate data associations
for split in ['train', 'test', 'valid']:
    print(f"Processing {split} data...")
    # Construct paths to the image and label folders for the current split
    image_folder = os.path.join(base_dir, split, 'images')
    label_folder = os.path.join(base_dir, split, 'labels')
    # Perform association and save the results
    data_associations[split] = associate_images_with_labels(image_folder, label_folder)

Processing train data...
Processing test data...
Processing valid data...


#Exploratory Data Analysis

In [11]:
print(len(data_associations['train']))
print(len(data_associations['test']))
print(len(data_associations['valid']))

2361
120
183


Observation: Data split isn't even

#Data Preprocessing

#Split the dataset

#Model Architecture